<h1>Google Geolocation API Notebook</h1>

<p>This notebook has been created to call the Google Geolocation API to convert the addresses that haven been retrieved during the web scraping into long and lat coordinates. These coordinates can the later be used for further analysis.<p> 

In [20]:
import pandas as pd

filepath = 'TablesDB/Location.csv'

df = pd.read_csv(filepath, delimiter=";")
df

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,NaN,NaN
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,NaN,NaN
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,NaN,NaN
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,NaN,NaN
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,NaN,NaN
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,NaN,NaN
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,NaN,NaN
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,NaN,NaN
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,NaN,NaN


<h2>Check the data for missing values</h2>

In [21]:
import pandas as pd
#counting missing addresses

missing_count = df['Street'].isnull().sum()
missing_zip = df['ZIP'].isnull().sum()
datapoints = df['LocationId'].count()

print("Total # of data points;", datapoints)
print("Missing addresses:", missing_count)
print("Missing ZIP codes:", missing_zip)

# Count the number of addresses without a street number
count_no_street_number = sum(~(df['Street'].str.match(r'^\d+.*\d+$') | df['Street'].str.match(r'^.*\d+$')))

print("Number of addresses without a street number:", count_no_street_number)




Total # of data points; 11352
Missing addresses: 708
Missing ZIP codes: 57
Number of addresses without a street number: 2377


<h2>Convert the addresses into lat/long coordinates using the Google Geocoding API</h2>

In [22]:
import json
import requests
import pandas as pd

api_key = #enter API key


# Request geolocation API


def get_geolocation(address, api_key):
    params = {
        'key': api_key,
        'address': address,
    }
    base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'

    # Server response
    response = requests.get(base_url, params=params)
    data = response.json()
    if data['status'] == 'OK':
        location = data['results'][0]['geometry']['location']
        return location['lat'], location['lng']
    else:
        return None, None

data = df



# Iterate through each address
for index, row in data.iterrows():
    # Concatenate street and ZIP columns to form the address
    address = f"{row['Street']}, {row['ZIP']}"

    # Get geolocation data
    latitude, longitude = get_geolocation(address, api_key)

    # Update Longitude and Latitude columns in the DataFrame
    data.at[index, 'latitude'] = latitude
    data.at[index, 'longitude'] = longitude

filepath_csv = 'TablesDB/Location_v2.csv'
data.to_csv(filepath_csv, index=False)
    






<h2>Check the final result</h2>

In [24]:
import pandas as pd

filepath = 'TablesDB/Location_v2.csv'

df2 = pd.read_csv(filepath, delimiter=",")
df2

,LocationId,Street,ZIP,longitude,latitude
0,78753131-6d78-4d1e-a0e5-fc8b5f430570,NaN,6598.0,NaN,NaN
1,f0c2bc0e-55ab-4eb1-98fa-edcb4a0ee01e,via albonago 43,6962.0,8.974113,46.013360
2,fbdcde66-0fd7-4304-a83e-70ecbb0f6ae7,Via San Gottardo 41,6500.0,9.268200,45.583280
3,2d7451a3-d7e1-4dc7-babf-ab237f2c02e4,Via F. Zorzi 17,6500.0,9.015443,46.187900
4,2bde1434-f480-4d46-9b05-99e0c0659671,Via San Gottardo,6900.0,NaN,NaN
...,...,...,...,...,...
11347,2dba2402-d23f-4ab8-9bb6-702bbe28cb4e,Chemin de la Petite-Californie,1222.0,6.207831,46.242178
11348,daed4779-995c-4c34-85e7-4b269d8bf5cc,Chemin William-Barbey,1292.0,6.149006,46.241745
11349,faddd38c-e9fc-4f4d-bf09-a41818cc620f,Rue du Temple 5,1236.0,2.352571,48.857414
11350,84d332ee-e1a8-4b5f-a432-82819fffe330,Chemin de la Pierre-à-Bochet,1226.0,6.214858,46.199271


In [25]:
import pandas as pd
#counting missing addresses

missing_long = df2['longitude'].isnull().sum()
missing_lat = df2['latitude'].isnull().sum()
datapoints = df2['LocationId'].count()

print("Total # of data points;", datapoints)
print("Missing long:", missing_long)
print("Missing lat:", missing_lat)

# Count the number of addresses without a street number
count_no_street_number = sum(~(df2['Street'].str.match(r'^\d+.*\d+$') | df2['Street'].str.match(r'^.*\d+$')))

print("Number of addresses without a street number:", count_no_street_number)

Total # of data points; 11352
Missing long: 2136
Missing lat: 2136
Number of addresses without a street number: 2377
